# Оптимизаторы 

---

## Классический градиентный спуск

Классический градиентный спуск вычисляет градиент функции потерь __по всей выборке__ и обновляет набор параметров $\theta$:

$$
\theta := \theta − \alpha \nabla_\theta J(\theta)
$$

Псевдокод:

```python
theta += -learning_rate * d_theta
```

При достаточно малой величине скорости обучения гарантированно делает шаг в сторону минимума.

In [2]:
from keras.optimizers import SGD

## Стохастический градиентный спуск

Стохастический градиентный спуск вычисляет градиент функции потерь __по маленькой части выборки (вплоть до одного экземпляра выборки) - батчу__ и обновляет набор параметров $\theta$:

$$
\theta := \theta − \alpha \nabla_\theta J(\theta, x^{(i:i+n)}, y^{(i:i+n)})
$$
, в частном случае:
$$
\theta := \theta − \alpha \nabla_\theta J(\theta, x^{(i)}, y^{(i)})
$$

Единственный разумный способ обрабатывать большие объемы данных, так как весь датасет, скорее всего, просто разом не поместится в оперативную память. Тем более это актуально при работе с изображениями. 

Кроме того, он используется для дообучения, когда нейронная сеть обучается уже во время своей работы (так называемое __онлайн-обучение__).

Без минусов тоже не обошлось: использование такого рода оптимизатора не гарантирует каждый раз четкий шаг в сторону минимума функции потерь, а потому и обучение может проходить не так плавно, как в случае классического градиентного спуска, а с флуктуациями.

<img src="images/sgd_fluctuation.png">

Чтобы не путаться, давайте говорить модно и молодёжно! 

* ___"Стохастический градиентный спуск"___ или ___"эс-гэ-дэ"___ предполагает, что обучение происходит по 1 экземпляру. 
* ___"Мини-батч градиент дэсэнт"___ или просто ___"батч СГД"___ предполагает, что обучение происходит по $n$ экземпляров.

In [1]:
from keras.optimizers import SGD

## Импульс

Импульсное обновление (англ. "momentum update") - немного другой подход, который тоже ускоряет сходимость в глубоких сетях. Данный подход возник от физического представления оптимизационной проблемы. Текущее значение параметров - условное положение шарика, катающегося по поверхности функции потерь, как по холмам и ямкам. На него действует гравитация, и поэтому он ускоряется на склонах и замедляется, попадая в ямки. Интуитивно предполагается, что его импульс поможет ему достичь искомой точки минимума быстрее.

$$ v_t = \gamma v_{t-1} + \alpha \nabla_\theta J(\theta) $$
$$ \theta := \theta - v_t $$

В терминах псевдокода:

```python
v = mu * v - learning_rate * d_theta  # изменение скорости
theta += v # изменение положения
```

<img src="images/with_without_momentum.png" width='50%'>
На рисунке выше показано поведение SGD __без__ _(слева)_ механизма импульса и __с__ _(справа)_.

In [3]:
from keras.optimizers import SGD

In [4]:
op = SGD(momentum=0.9)

## Импульс Нестерова

Импульс Нестерова (англ. "Nesterov momentum", он же "Nesterov accelerated gradient", или "NAG"). Говорят, что работает лучше! Но как?

Давайте представим, что у нас уже есть импульс, и наш мячик катится. Обновление градиента по схеме выше состоит из двух компонент: непосредственно импульса и градиента. Тогда перед вычислением значения градиента мы можем заранее предсказать, куда катится шарик: это `mu + v`-часть в формуле выше (в англицкой литературе это называется _lookahead-vector_). И тогда мы можем вычислить градиент не от текущего положения, а от этого предсказанного, lookahead.

<img src='images/nesterov.png' width='70%'>

$$ v_t = \gamma v_{t-1} + \alpha \nabla_\theta J(\theta - \gamma v_{t-1})  $$
$$ \theta := \theta - v_t $$

Псевдокод:

```python
theta_ahead = theta + mu * v
# вот здесь вычисляем градиент не от текущего положения theta, а от lookahead
v = mu * v - learning_rate * d_theta_ahead  
theta += v
```

In [5]:
from keras.optimizers import SGD

In [6]:
op = SGD(nesterov=True)

## тлен

Чем дольше идет процесс обучения, тем _вроде бы_ мы ближе стремимся к правильному решению, минимуму нашей функции потерь. А если это так, то необходимо со временем охлаждать наши темпы, постепенно уменьшая _скорость обучения $\alpha$_. 

Этот механизм называется __learning rate decay__, или _.."угасание"? "тление"?..  скорости обучения_.

Вариантов угасания много, но по умолчанию `SGD` из keras поддерживает один: линейное угасание с каждой эпохой.

In [7]:
op = SGD(decay=0.00001)

---

## Adagrad

Во всех предыдущих случаях мы меняли обновляли все параметры глобально и разом. Это не всегда, однако, является лучшим решением.

Давайте посмотрим на модификацию SGD, которую предложили [Duchi и его команда](http://jmlr.org/papers/v12/duchi11a.html).

```python
cache += d_theta ** 2  # cache - вектор с обновлениями параметров
theta += -learning_rate * d_theta / (sqrt(cache) + eps)
```

Используемый `cache` используется после своего вычисления для того, чтобы в дальнейшем нормализовать обновления параметров theta. Если обновления некоторого параметра из вектора theta происходят часто, то благодаря делению на корень из `cache`, каждое из обновлений будет небольшим. И наоборот, редкие обновления будут осуществляться с меньшим штрафом.

`eps` $= 10^{-4}...10^{-8}$ - использована, чтобы не разделить случайно на ноль.

Минус Adagrad в том, что он слишком жестко замедляет обновление, и обучение тормозится слишком рано.

In [8]:
from keras.optimizers import Adagrad

In [9]:
op = Adagrad(epsilon=0.00001)

## RMSProp

RMSProp - попытка решения проблемы агрессивности Adagrad, и достаточно эффективная. Был использован механизм скользящего среднего, который может быть записан так:

```python
cache = decay_rate * cache + (1 - decay_rate) * d_theta**2
theta += - learning_rate * d_theta / (np.sqrt(cache) + eps)
```

## Adam

Предложенный относительно недавно метод. Его обновление похоже на обновление в RMSProp, но обновления еще более гладкие.

```python
m = beta1*m + (1-beta1)*d_theta
v = beta2*v + (1-beta2)*(d_theta**2)
theta += -learning_rate * m / (np.sqrt(v) + eps)
```

Рекомендованные авторами значения: `eps`$=10^{-8}$, `beta1`$=0.9$, `beta2`$=0.999$.

# Как это выглядит?

https://github.com/Jaewan-Yun/optimizer-visualization

Проследите внимательно, как ведет себя в разных ситуациях каждый. Сравните друг с другом.

# Какой выбрать?

На практике рекомендуется пробовать прежде всего __Adam__, и __SGD + Nesterov Momentum__ как альтернативу.